# Zuora/Eloqua export/import service

Everything you need to know about migrating data from the Zuora subscription service to the Eloqua marketing service.


Zuora export service, responsible for exporting any object SQL-style call in CSV/JSON format
More on that here: https://knowledgecenter.zuora.com/DC_Developers/M_Export_ZOQL

zuora export service?


In [ ]:
var importer = require('../Core');
var xlsx = require('xlsx');
var request = importer.import('request polyfill');

function getAuthHeaders(zuoraConfig) {
    return {
        'Content-Type': 'application/json',
        'apiAccessKeyId': zuoraConfig.rest_api_user,
        'apiSecretAccessKey': zuoraConfig.rest_api_password,
        'Accept': 'application/json'
    };
}

function createBulkExportJob(query, zuoraConfig) {
    return request.request({
        followAllRedirects: true,
        uri: zuoraConfig.rest_api_url + '/object/export',
        json: query,
        method: 'POST',
        headers: getAuthHeaders(zuoraConfig)
    }).then(r => r.body.Id)
}

function getBulkExportJobStatus(exportId, zuoraConfig) {
    console.log('waiting...');
    return request.request({
        followAllRedirects: true,
        uri: zuoraConfig.rest_api_url + '/object/export/' + exportId,
        method: 'GET',
        headers: getAuthHeaders(zuoraConfig)
    }).then(r => {
        if (r.body.Status === 'Completed') {
            return r.body.FileId;
        } else if (r.body.Status === 'Processing' || r.body.Status === 'Pending') {
            return new Promise(resolve => setTimeout(resolve, 500))
                .then(() => getBulkExportJobStatus(exportId, zuoraConfig));
        } else {
            throw new Error('Export status error ' + r.statusCode + ' ' + r.body.Status);
        }
    });
}

function getBulkExportFile(fileId, zuoraConfig) {
    return request.request({
        followAllRedirects: true,
        uri: zuoraConfig.rest_api_url + '/files/' + fileId,
        method: 'GET',
        headers: getAuthHeaders(zuoraConfig)
    }).then(r => r.body)
}

function csvToJson(csv) {
    const workbook = xlsx.read(new Buffer(csv), {type:"buffer"});
    return xlsx.utils.sheet_to_json(workbook.Sheets[workbook.SheetNames[0]]);
}

module.exports = {
    csvToJson,
    createBulkExportJob,
    getBulkExportFile,
    getBulkExportJobStatus,
    getAuthHeaders
}


zuora export catalog


In [ ]:

function getCatalog(zuoraConfig, next) {
    var catalog = [];
    return request.request({
        followAllRedirects: true,
        uri: zuoraConfig.rest_api_url + (typeof next !== 'undefined' ? next.replace(/\/v1/ig, '') : '/catalog/products'),
        method: 'GET',
        headers: getAuthHeaders(zuoraConfig)
    }).then(r => {
        catalog = catalog.concat(r.body.products);
        if(r.body.nextPage) {
            return getCatalog(zuoraConfig, r.body.nextPage).then(r => catalog.concat(r));
        }
        return catalog;
    })
}
module.exports = getCatalog;


zuora export service test?


In [ ]:
var assert = require('assert');
var sinon = require('sinon');
var importer = require('../Core');
var {
    createBulkExportJob,
    getBulkExportJobStatus,
    getBulkExportFile,
    csvToJson,
} = importer.import('zuora to eloqua.ipynb[0]');
var request = importer.import('http request polyfill');

var sandbox = sinon.createSandbox();
var zuoraConfig = {
    "rest_api_user":"devteam@fakepage.com",
    "rest_api_password":"pass",
    "rest_api_url": "http://localhost:18888"
};

describe('zuora oauth', () => {
    
    afterEach(() => {
        sandbox.restore();
    })
    
    it('should connect to zuora using oauth', () => {
        const dummyId = '123';
        const dummyQuery = 'SELECT * FROM Accounts';
        
        const requestStub = sandbox.stub(request, "request")
            .returns(Promise.resolve({ body: {Id: dummyId } }));
        
        return createBulkExportJob(dummyQuery, zuoraConfig)
            .then(result => {
                assert.equal(result, dummyId);
                assert(requestStub.calledOnce, 'request should only be called once');
                const stubCall = requestStub.getCall(0);
                assert.equal(stubCall.args[0].json, dummyQuery);
            });
    })
    
    it('should wait for the export to complete', () => {
        const dummyId = '12345'
        
        const requestStub = sandbox.stub(request, "request")
            .returns(Promise.resolve({ body: {Status: 'Completed', FileId: dummyId } }));
        
        return getBulkExportJobStatus('123', zuoraConfig)
            .then(result => {
                assert.equal(result, dummyId);
                assert(requestStub.calledOnce, 'request should only be called once');
            })
    })
    
    it('should download the csv file', () => {
        const csvFile = 'some,csv,file';
        
        const requestStub = sandbox.stub(request, "request")
            .returns(Promise.resolve({ body: csvFile }));
        
        return getBulkExportFile('1234', zuoraConfig)
            .then(result => {
                assert.equal(result, csvFile);
                assert(requestStub.calledOnce, 'request should only be called once');
            })
    })
    
    it('should convert CSV to JSON', () => {
        const result = csvToJson('some,csv,file\n1,2,3');
        assert.equal(result[0].file, 3);
    })
    
})



zuora renewals query?

Zuora query, an example query that retrieves the entire list of zuora subscriptions between two dates (start, end)
The resulting JSON is passed directly to the API: https://www.zuora.com/developer/api-reference/#operation/Object_POSTExport



In [ ]:
var moment = require('moment');
var chrono = require('chrono-node');
var excludedRatePlans = [
    'Act! Pro - New License',
    'Act! Pro - 30 Day Support',
    'Act! Pro - Upgrade License',
    'Act! Password Reset Charge',
    'Act! Premium Cloud - Trial',
    'Act! Pro V19 - Upgrade License',
    'Act! Pro V20 - Upgrade License',
]
var excludedProductSkus = [
    '00000006'
]
var currencies = [
    '',
    'USD',
    'AUD',
    'NZD',
]

var query = `SELECT
    Account.Id,
    Account.Name,
    Account.AccountNumber,
    Account.resellerofRecord__c,
    Account.renewalRep__c,
    Account.commisionedSalesRep__c,
    Account.CreatedDate,
    Account.Currency,
    SoldToContact.WorkEmail,
    SoldToContact.Country,
    SoldToContact.State,
    BillToContact.WorkEmail,
    RatePlan.Id,
    RatePlan.Name,
    RatePlanCharge.Id,
    RatePlanCharge.BillingPeriod,
    RatePlanCharge.Description,
    RatePlanCharge.Quantity,
    RatePlanCharge.Version,
    RatePlanCharge.CreatedDate,
    RatePlanCharge.EffectiveEndDate,
    DefaultPaymentMethod.CreditCardExpirationMonth,
    DefaultPaymentMethod.CreditCardExpirationYear,
    DefaultPaymentMethod.CreditCardMaskNumber,
    ProductRatePlanCharge.Id,
    ProductRatePlan.planType__c,
    ProductRatePlan.planSubType__c,
    ProductRatePlan.Id,
    ProductRatePlan.Name,
    Product.productType__c,
    Product.Name,
    Product.Description,
    Product.Id,
    Product.SKU,
    Subscription.Id,
    Subscription.Name,
    Subscription.Status,
    Subscription.Reseller__c,
    Subscription.SubscriptionEndDate,
    Subscription.SubscriptionStartDate,
    Subscription.TermStartDate,
    Subscription.TermEndDate,
    Subscription.AutoRenew
FROM RatePlanCharge
WHERE Subscription.Status!='Draft' AND Subscription.Status!='Cancelled' AND Subscription.Status!='Expired'
    AND Subscription.TermEndDate &gt;='{0}' AND Subscription.TermEndDate &lt;='{1}'
    AND (Account.Currency='${currencies.join("' OR Account.Currency='")}')
    AND (ProductRatePlan.Name!='${excludedRatePlans.join("' AND ProductRatePlan.Name!='")}')
    AND (Product.SKU!='${excludedProductSkus.join("' AND Product.SKU!='")}')
    AND NOT (SoldToContact.WorkEmail LIKE 'qaaw%@gmail.com')
    AND NOT (BillToContact.WorkEmail LIKE 'qaaw%@gmail.com')
    AND NOT (Account.Name LIKE '%do not use%')
`;
// AND (RatePlanCharge.EffectiveEndDate &gt;='{2}' OR RatePlanCharge.ChargeType='OneTime')
// removed this so that discounts show up on the account
// AND RatePlanCharge.BillingPeriod!='Month'

function getQuery(start, end) {
    // TODO: add option for pulling based on subscription term or based on modified fields
    return {
        Query: query.replace('{0}', moment(chrono.parseDate(start)).format('YYYY-MM-DD'))
                    .replace('{1}', moment(chrono.parseDate(end)).format('YYYY-MM-DD'))
                    .replace('{2}', moment(new Date()).format('YYYY-MM-DD')),
        Format: 'csv',
        Zip: false
    };
}
module.exports = {
    getQuery
};


test zuora renewals query?


In [ ]:
var assert = require('assert');
var importer = require('../Core');
var renewalsQuery = importer.import('zuora renewals query');

describe('zuora query', () => {
    it('should include the dates specified', () => {
        const now = new Date();
        const year = now.getMonth() < 11 ? (now.getFullYear() - 1) : now.getFullYear()
        const q = renewalsQuery.getQuery('beginning of November', 'beginning of December');
        assert(q.Query.includes(year + '-11-01'), 'should have correct dates');
    })
})


eloqua import service?

eloqua create template, this template should match the the output from the eloqua customobject/:id/fields REST call:
More on that here: https://docs.oracle.com/cloud/latest/marketingcs_gs/OMCAC/api-Application-2.0-Custom%20object%20data.html



In [ ]:
esvar importer = require('../Core');
var request = importer.import('http request polyfill');
var {
    bulkImportTemplate,
    temporaryImportTemplate
} = importer.import('eloqua create template');

function eloquaOauth(eloquaConfig) {
    if (typeof eloquaConfig === 'undefined'
        || eloquaConfig === null
        || typeof eloquaConfig.rest_api_company === 'undefined'
        || typeof eloquaConfig.rest_api_user === 'undefined'
        || typeof eloquaConfig.rest_api_password === 'undefined'
        || typeof eloquaConfig.rest_client_id === 'undefined'
        || typeof eloquaConfig.rest_secret === 'undefined') {
        throw new Error('Please supply valid config eloqua configuration.');
    }
    var authBody = {
        "grant_type": "password",
        "scope": "full",
        "username": eloquaConfig.rest_api_company + '\\' + eloquaConfig.rest_api_user,
        "password": eloquaConfig.rest_api_password
    };
    return request.request({
        followAllRedirects: true,
        uri: eloquaConfig.token_uri,
        method: 'POST',
        json: authBody,
        headers: {
            'Authorization': "Basic " + new Buffer(eloquaConfig.rest_client_id + ":" + eloquaConfig.rest_secret).toString("base64"),
            'Content-Type': 'application/json',
            'Accept': 'application/json'
        }
    }).then(res => {
        res.body.expires = (new Date()).getTime() + parseFloat(res.body.expires_in) * 1000;
        return res.body;
    });
}

function eloquaRequestHeaders(eloquaToken) {
    return {
        'Authorization': "Bearer " + eloquaToken.access_token,
        'Content-Type': 'application/json',
        'Accept': 'application/json'
    };
}

function eloquaBulkImportStatus(syncUri, eloquaToken, eloquaConfig) {
    console.log(syncUri);
    return request.request({
        followAllRedirects: true,
        uri: eloquaConfig.rest_api_url + '/bulk/2.0' + syncUri,
        method: 'GET',
        headers: eloquaRequestHeaders(eloquaToken)
    }).then(res => {
        if (res.body.status === 'success' || res.body.status === 'warning') {
            return true;
        } else if (res.body.status === 'active' || res.body.status === 'pending') {
            return new Promise(resolve => setTimeout(resolve, 500))
                .then(() => eloquaBulkImportStatus(syncUri, eloquaToken, eloquaConfig));
        } else {
            throw new Error('Sync status error ' + res.statusCode + ' ' + JSON.stringify(res.body));
        }
    });
}

function completeBulkImportSync(importUri, eloquaToken, eloquaConfig) {
    return request.request({
        followAllRedirects: true,
        uri: eloquaConfig.rest_api_url + '/bulk/2.0/syncs',
        method: 'POST',
        json: {
            syncedInstanceUri: importUri
        },
        headers: eloquaRequestHeaders(eloquaToken)
    }).then(res => {
        const syncUri = res.body.uri;
        return eloquaBulkImportStatus(syncUri, eloquaToken, eloquaConfig);
    });
}

function startBulkImportData(json, importUri, eloquaToken, eloquaConfig) {
    return request.request({
        followAllRedirects: true,
        uri: eloquaConfig.rest_api_url + '/bulk/2.0' + importUri + '/data',
        method: 'POST',
        json: json,
        headers: eloquaRequestHeaders(eloquaToken)
    });
}

// https://docs.oracle.com/cloud/latest/marketingcs_gs/OMCAB/Developers/BulkAPI/Endpoints/Custom%20objects/Imports/post-customObjects-imports.htm
function createImportDefinition(customDataObjectId, eloquaToken, eloquaConfig) {
    return request.request({
        followAllRedirects: true,
        uri: eloquaConfig.rest_api_url + '/bulk/2.0/customobjects/' + customDataObjectId + '/imports',
        method: 'POST',
        json: bulkImportTemplate(customDataObjectId),
        headers: eloquaRequestHeaders(eloquaToken)
    }).then(res => {
        return res.body.uri;
    });
}

function createInstanceDefinition(instanceId, executionId, eloquaToken, eloquaConfig) {
    return request.request({
        followAllRedirects: true,
        uri: eloquaConfig.rest_api_url + '/bulk/2.0/contacts/imports',
        method: 'POST',
        json: temporaryImportTemplate(instanceId, executionId),
        headers: eloquaRequestHeaders(eloquaToken)
    }).then(res => {
        return res.body.uri;
    });
}

module.exports = {
    eloquaOauth,
    createImportDefinition,
    startBulkImportData,
    completeBulkImportSync,
    eloquaBulkImportStatus,
    createInstanceDefinition,
    eloquaRequestHeaders
}


test eloqua import service?


In [ ]:
var assert = require('assert');
var sinon = require('sinon');
var importer = require('../Core');
var request = importer.import('http request polyfill');
var {
    eloquaOauth,
    createImportDefinition,
    startBulkImportData,
    completeBulkImportSync
} = importer.import('eloqua import service');

var {
    getEloquaConfig,
    getOauthToken,
    getImportData
} = importer.import('eloqua import blueprints');

var eloquaConfig = getEloquaConfig();
var eloquaToken = getOauthToken();
var sandbox = sinon.createSandbox();

describe('eloqua bulk upload', () => {
        
    afterEach(() => {
        sandbox.restore();
    })

    it('should get a valid oauth token', () => {
        
        const requestStub = sandbox.stub(request, "request")
            .returns(Promise.resolve({ body: {expires_in: 1000 } }));
        
        return eloquaOauth(eloquaConfig)
            .then(r => {
                assert(r.expires > (new Date()).getTime());
                assert(requestStub.calledOnce, 'request should only be called once');
            })
    })
    
    it('should create a bulk import instance', () => {
        
        const requestStub = sandbox.stub(request, "request")
            .returns(Promise.resolve({ body: {uri: '/imports/123' } }));
        
        return createImportDefinition(60, eloquaToken, eloquaConfig)
            .then(r => {
                assert(r.includes('/imports/'));
                assert(requestStub.calledOnce, 'request should only be called once');
            })
    })
    
    it('should update data to eloqua', () => {
        
        const requestStub = sandbox.stub(request, "request")
            .returns(Promise.resolve({ body: {uri: '/imports/123' }, statusCode: 204 }));
        
        return startBulkImportData([getImportData()], '/imports/123', eloquaToken, eloquaConfig)
            .then(r => {
                assert(r.statusCode === 204, 'invalid status recieved from import ' + r.statusCode);
                assert(requestStub.calledOnce, 'request should only be called once');
            })
    })
    
    it('should verify upload was successful', () => {
        var importUri;
        
        const requestStub = sandbox.stub(request, "request")
            .returns(Promise.resolve({ body: {uri: '/imports/123', status: 'success' } }));
        
        return completeBulkImportSync(importUri, eloquaToken, eloquaConfig)
            .then(r => {
                assert(r === true);
                assert(requestStub.called, 'request should only be called once');
            })
    })
    
})


zuora eloqua mapper?



In [ ]:
var moment = require('moment');
var _ = require('lodash');
var chrono = require('chrono-node');

function mapRatePlanToProduct(description) {
    if(description.includes('trial') > -1)
        return 'trial';
    else if (description.includes('volume') || description.includes('discount'))
        return 'discount';
    else if (description.includes('cloud') && !description.includes('trial'))
        return 'actpremiumcloud';
    else if (description.includes('premium') && !description.includes('cloud')
             && !description.includes('trial') && !description.includes('support'))
        return 'actpremium';
    else if (description.includes('pro')  && !description.includes('support'))
        return 'actpro';
    else if (description.includes('support'))
        return 'support';
    else if (description.includes('handheld'))
        return 'handheld';
    else if (description.includes('aem'))
        return 'aem';
    else throw new Error('product not recognized ' + description);
}

function mapDataToFields(records) {
    var uniqueIds = _.groupBy(records, r => r['Account.Id']);
    return Object.keys(uniqueIds).map(k => {
        const rpcs = _.groupBy(uniqueIds[k], r => r['RatePlanCharge.Id']);
        const charges = Object.keys(rpcs).map(k => _.sortBy(rpcs[k], r => r['RatePlanCharge.Version']).pop());
        const record = {};
        charges.sort((a, b) =>
                          chrono.parseDate(b['Subscription.TermEndDate']).getTime()
                          - chrono.parseDate(a['Subscription.TermEndDate']).getTime());
        // contact information
        const contact = charges.filter(charge => charge['SoldToContact.WorkEmail'] || charge['BillToContact.WorkEmail'])[0]
        if(typeof contact === 'undefined') {
            console.log(charges);
            return;
        }
        record['EmailAddress'] = contact['SoldToContact.WorkEmail'] || contact['BillToContact.WorkEmail'];
        record['State'] = contact['SoldToContact.State'];
        record['Country'] = contact['SoldToContact.Country'];
        record['Currency'] = contact['Account.Currency'];

        // primary product on subscription
        const actProduct = charges.filter(p => {
            const pname = mapRatePlanToProduct(p['ProductRatePlan.Name'].toLowerCase());
            return pname === 'actpremiumcloud' || pname === 'actpremium' || pname === 'actpro' || pname === 'trial'
        })[0];
        
        if(typeof actProduct !== 'undefined') {
            record['ActProduct'] = mapRatePlanToProduct(actProduct['ProductRatePlan.Name'].toLowerCase());
            record['Quantity'] = actProduct['RatePlanCharge.Quantity'];
        } else {
            record['ActProduct'] = 'Unknown';
            record['Quantity'] = 0;
        }
        
        // discounts!
        const discount = charges.filter(p => {
            const pname = mapRatePlanToProduct(p['ProductRatePlan.Name'].toLowerCase());
            return pname === 'discount'
        })[0];
        if(typeof discount !== 'undefined') {
            record['Discount'] = actProduct['ProductRatePlan.Name'];
        } else {
            record['Discount'] = '';
        }
        
        // support
        const support = charges.filter(charge => {
            return mapRatePlanToProduct(charge['ProductRatePlan.Name'].toLowerCase()) === 'support'
        })[0];
        if(typeof support !== 'undefined') {
            record['Support'] = mapRatePlanToProduct(support['ProductRatePlan.Name'].toLowerCase());
            record['SupportQuantity'] = support['RatePlanCharge.Quantity'];
        } else {
            record['Support'] = 'Unknown';
            record['SupportQuantity'] = 0;
        }
        
        // subscription data
        const renewal = chrono.parseDate(charges[0]['Subscription.TermEndDate']);
        record['RenewalsStatus'] = charges[0]['Subscription.Status'];
        record['RenewalDate'] = moment(renewal).format('YYYY-MM-DD');
        
        // card expiration
        const expiration = new Date();
        expiration.setDate(1);
        expiration.setMonth(parseInt(charges[0]['DefaultPaymentMethod.CreditCardExpirationMonth']) - 1);
        expiration.setYear(parseInt(charges[0]['DefaultPaymentMethod.CreditCardExpirationYear']));
        record['CardExpiration'] = moment(expiration).format('YYYY-MM-DD');
        //record['Last4DigitsOfCard'] = ((/([0-9]+)/ig).exec(charges[0]['DefaultPaymentMethod.CreditCardMaskNumber']) || [])[1];
        
        // account data
        record['RepName'] = charges[0]['Account.renewalRep__c'];
        record['RORName'] = charges[0]['Account.resellerofRecord__c'];
        record['RORNumber'] = ((/([0-9]+)/ig).exec(charges[0]['Account.resellerofRecord__c']) || [])[1];
        record['AccountId'] = charges[0]['Account.Id'];

        return record;
    }).filter(r => typeof r !== 'undefined');
}
module.exports = {
    mapDataToFields
};


zuora eloqua mapper test?


In [ ]:
describe('map zuora data fields', () => {
    it('should map basic data', () => {
        
    })
    
    it('should map contact data', () => {
        
    })
    
    it('should map support data', () => {
        
    })
    
    it('should map cancelled data', () => {
        
    })
})


eloqua import create template?


In [ ]:

module.exports = {
    bulkImportTemplate,
    contentCreateTemplate,
    temporaryImportTemplate
}

function bulkImportTemplate(templateId) {
    // replace ID for CDO
    return {
        "name": "Renewals Micro-service - Bulk Import",
        "mapDataCards": "true",
        "mapDataCardsEntityField": "{{Contact.Field(C_EmailAddress)}}",
        "mapDataCardsSourceField": "EmailAddress",
        "mapDataCardsEntityType": "Contact",
        "mapDataCardsCaseSensitiveMatch": "false",
        "updateRule": "always",
        "fields": {
            "AccountId": `{{CustomObject[${templateId}].Field(Account_ID1)}}`,
            "ActProduct": `{{CustomObject[${templateId}].Field(Act_Product1)}}`,
            "EmailAddress": `{{CustomObject[${templateId}].Field(Email_Address1)}}`,
            "Quantity": `{{CustomObject[${templateId}].Field(Quantity1)}}`,
            "Support": `{{CustomObject[${templateId}].Field(Support1)}}`,
            "SupportQuantity": `{{CustomObject[${templateId}].Field(Support_Quantity1)}}`,
            "RenewalDate": `{{CustomObject[${templateId}].Field(Renewal_Date1)}}`,
            "RenewalsStatus": `{{CustomObject[${templateId}].Field(Renewal_Status1)}}`,
            "RepName": `{{CustomObject[${templateId}].Field(Rep_Name1)}}`,
            "RORName": `{{CustomObject[${templateId}].Field(ROR_Name1)}}`,
            "RORNumber": `{{CustomObject[${templateId}].Field(ROR_Number1)}}`,
            "CardExpiration": `{{CustomObject[${templateId}].Field(Card_Expiration1)}}`,
            "State": `{{CustomObject[${templateId}].Field(State1)}}`,
            "Country": `{{CustomObject[${templateId}].Field(Country1)}}`,
            "Currency": `{{CustomObject[${templateId}].Field(Currency1)}}`
        },
        "identifierFieldName": "EmailAddress"
    }
}

function contentCreateTemplate() {
    return {
        "recordDefinition": {
            "ContactID": "{{Contact.Id}}",
            "EmailAddress": "{{Contact.Field(C_EmailAddress)}}"
        },
        "height": 256,
        "width": 256,
        "editorImageUrl": "https://purchasesprint.actops.com/assets/act-logo-circle.png",
        "requiresConfiguration": false,
    }
}

function temporaryImportTemplate(instance, execution) {
    return {
        "name": "Renewals Micro-service - Bulk Import",
        "mapDataCards": "true",
        "mapDataCardsEntityField": "{{Contact.Field(C_EmailAddress)}}",
        "mapDataCardsSourceField": "EmailAddress",
        "mapDataCardsEntityType": "Contact",
        "mapDataCardsCaseSensitiveMatch": "false",
        "updateRule": "always",
        "fields": {
            "EmailAddress": "{{Contact.Field(C_EmailAddress)}}",
            "Last4DigitsOfCard": "{{Contact.Field(Last_4_Digits_of_Card1)}}",
            "Content": `{{ContentInstance(${instance}).Execution[${execution}]}}`
        },
        "identifierFieldName": "EmailAddress"
    }
}


test eloqua import create template?


In [ ]:
var assert = require('assert');
var importer = require('../Core');
var { temporaryImportTemplate } = importer.import('eloqua import create template');

describe('eloqua bulk import definition', () => {
    it('should return the current import definition', () => {
        const dummyInstance = 'instance123';
        const template = temporaryImportTemplate(dummyInstance, 'execution123');
        assert(JSON.stringify(template).includes(dummyInstance));
    })
})


eloqua import blueprints?



In [ ]:

function getImportData() {
    return {
        "AccountId": '1234',
        "ActProduct": 'premium',
        "EmailAddress": "brian.cullinan@swiftpage.com",
        "ExpirationMonth": '12',
        "ExpirationYear": '2017',
        "Last4DigitsOfCard": '1234',
        "Quantity": 5,
        "RenewalDate": '11/12/2017',
        "RenewalsStatus": "Drew loves marketing!",
        "RepName": 'Brian',
        "RORName": 'James',
        "RORNumber": '1111',
        "State": 'AZ',
    }
}

function getOauthToken() {
    return {
        "access_token": "access_token",
        "token_type": "bearer",
        "expires_in": 28800,
        "refresh_token": "refresh_token",
        "expires": (new Date()).getTime() + 28800
    }
}

function getEloquaConfig() {
    return {
        "authorize_uri":"http://localhost:18888/auth/oauth2/authorize",
        "token_uri":"http://localhost:18888/auth/oauth2/token",
        "rest_api_url":"http://localhost:18888",
        "rest_client_id":"client-id",
        "rest_secret":"secret",
        "rest_api_company": "swiftpage",
        "rest_api_user":"username",
        "rest_api_password":"password"
    }
}
module.exports = {
    getImportData,
    getOauthToken,
    getEloquaConfig
};


eloqua existing import?


In [ ]:
var importer = require('../Core');
var request = importer.import('http request polyfill');
var eloquaImport = importer.import('eloqua import service');
var { bulkImportTemplate } = importer.import('eloqua create template');

function getCustomDataObject(eloquaToken, eloquaConfig) {
    return request.request({
        followAllRedirects: true,
        uri: eloquaConfig.rest_api_url + '/bulk/2.0/customobjects', // /60/fields',
        method: 'GET',
        headers: eloquaImport.eloquaRequestHeaders(eloquaToken)
    })
        .then(r => r.body.items.filter(i => i.name === 'AUT - NA Renewals')[0])
}

function getImportDefinitions(uri, eloquaToken, eloquaConfig) {
    return request.request({
        followAllRedirects: true,
        uri: eloquaConfig.rest_api_url + '/bulk/2.0' + uri + '/imports',
        method: 'GET',
        headers: eloquaImport.eloquaRequestHeaders(eloquaToken)
    })
        .then(r => r.body.items.filter(i => i.name === bulkImportTemplate(0).name)[0])
}

module.exports = {
    getCustomDataObject,
    getImportDefinitions
};


test eloqua existing import?


In [ ]:
var assert = require('assert');
var sinon = require('sinon');
var importer = require('../Core');
var request = importer.import('http request polyfill');
var existing = importer.import('eloqua existing import');
var { bulkImportTemplate } = importer.import('eloqua create template');
var {
    getEloquaConfig,
    getOauthToken,
} = importer.import('eloqua import blueprints');

var eloquaConfig = getEloquaConfig();
var eloquaToken = getOauthToken();
var sandbox = sinon.createSandbox();

describe('eloqua existing import', () => {
        
    afterEach(() => {
        sandbox.restore();
    })

    it('should check if the custom object is configured', () => {
        
        const requestStub = sandbox.stub(request, "request")
            .returns(Promise.resolve({ body: {items: [{name: 'AUT - NA Renewals'}] } }));
        
        return existing.getCustomDataObject({}, eloquaConfig)
            .then(r => {
                assert(r);
                assert(requestStub.calledOnce, 'request should only be called once');
            })
    })
    
    it('should check if there is already an import definition', () => {
        
        const requestStub = sandbox.stub(request, "request")
            .returns(Promise.resolve({ body: {items: [{name: bulkImportTemplate(0).name}] } }));
        
        return existing.getImportDefinitions('/imports/1234', eloquaToken, eloquaConfig)
            .then(r => {
                assert(r);
                assert(requestStub.calledOnce, 'request should only be called once');
            })
    })
    
});


use the eloqua rest api to retrieve a list of objects


In [ ]:

if(typeof $$ !== 'undefined') {
    $$.async();
    var eloquaToken, eloquaConfig = JSON.parse(fs.readFileSync(PROFILE_PATH + '/.credentials/eloqua_production.json').toString().trim());
    eloquaOauth(eloquaConfig)
        .then(r => {
            eloquaToken = r;
            return getEloquaExistingImport(eloquaToken, eloquaConfig)
        })
        /*
        .then(r => {
            // delete import definitions
            const imports = JSON.parse(r.body).items;
            return importer.runAllPromises(imports.map(r => resolve => {
                return request({
                    followAllRedirects: true,
                    uri: eloquaConfig.rest_api_url + '/bulk/2.0' + r.uri,
                    method: 'DELETE',
                    headers: {
                        'Authorization': "Bearer " + eloquaToken.access_token,
                        'Content-Type': 'application/json',
                        'Accept': 'application/json'
                    }
                }).then(r => resolve(r)).catch(e => resolve(e))
            }))
        })
        */
        .then(r => {
            console.log(r);
            $$.mime({'text/html': '<pre>' + JSON.stringify(r, null, 4) + '</pre>'});
        })
        .catch(e => $$.sendError(e))
}

// TODO: find other definitions, compare, and import using the same definition



aws entry point?

In [ ]:
var importer = require('../Core');
var eloquaUpload = importer.import('bulk upload eloqua');
var zuoraExport = importer.import('zuora export month');
var mapper = importer.import('zuora eloqua mapper');

function handler(event, context, callback) {
    const zuoraConfig = {
        "rest_api_user": process.env.ZUORA_API_USER,
        "rest_api_password": process.env.ZUORA_API_PASS,
        "rest_api_url": process.env.ZUORA_API_URL
    };
    const eloquaConfig = {
        "authorize_uri": process.env.ELOQUA_AUTHORIZE_URL,
        "token_uri": process.env.ELOQUA_TOKEN_URL,
        "rest_api_url": process.env.ELOQUA_API_URL,
        "rest_client_id": process.env.ELOQUA_CLIENT_ID,
        "rest_secret": process.env.ELOQUA_CLIENT_SECRET,
        "rest_api_company": process.env.ELOQUA_API_COMPANY,
        "rest_api_user": process.env.ELOQUA_API_USER,
        "rest_api_password": process.env.ELOQUA_API_PASS
    };
    return zuoraExport.getZuoraMonth(0, zuoraConfig)
        .then(records => mapper.mapDataToFields(records))
        .then(accounts => eloquaUpload.bulkUploadEloqua(accounts, eloquaConfig))
        .then(() => callback(null, {
            'statusCode': 200,
            'headers': { 'Content-Type': 'text/plain' },
            'body': 'Success!'
        }))
        .catch(e => callback(e, {
            'statusCode': 500,
            'headers': { 'Content-Type': 'text/plain' },
            'body': 'Error: ' + e.message
        }))
}
module.exports = {
    handler
};


test aws entry point?


In [ ]:
var assert = require('assert');
var sinon = require('sinon');
var importer = require('../Core');
var bundle = importer.import('aws entry point');
var eloquaUpload = importer.import('bulk upload eloqua');
var zuoraExport = importer.import('zuora export month');
var mapper = importer.import('zuora eloqua mapper');

process.env.ZUORA_API_USER = 'devteam@fakepage.com';
process.env.ZUORA_API_USER = 'pass';
process.env.ZUORA_API_USER = 'http://localhost:18888';
var sandbox = sinon.createSandbox();

describe('bulk upload entry point', () => {
    
    afterEach(() => {
        sandbox.restore();
    })
    
    it('should call zuora month export', () => {
        var callback = sinon.spy();
        sandbox.stub(zuoraExport, 'getZuoraMonth').returns(Promise.resolve([]));
        return bundle.handler({}, null, callback)
            .then(() => {
                assert(zuoraExport.getZuoraMonth.calledWithMatch(0));
            });
    });
    
    it('should call data mapper', () => {
        var callback = sinon.spy();
        sandbox.stub(zuoraExport, 'getZuoraMonth').returns(Promise.resolve([]));
        sandbox.stub(mapper, 'mapDataToFields').returns([]);
        return bundle.handler({}, null, callback)
            .then(() => {
                assert(mapper.mapDataToFields.calledWithMatch([]));
            });
    });
    
    it('should call bulk upload', () => {
        var callback = sinon.spy();
        sandbox.stub(zuoraExport, 'getZuoraMonth').returns(Promise.resolve([]));
        sandbox.stub(mapper, 'mapDataToFields').returns([]);
        sandbox.stub(eloquaUpload, 'bulkUploadEloqua').returns([]);
        return bundle.handler({}, null, callback)
            .then(() => {
                assert(eloquaUpload.bulkUploadEloqua.calledWithMatch([]));
            });
    });
})


notify entry point?


In [ ]:
var importer = require('../Core');
var zuoraExport = importer.import('zuora account service');
var eloquaUpload = importer.import('bulk upload eloqua');

function handler(event, context, callback) {
    const zuoraConfig = {
        "rest_api_user": process.env.ZUORA_API_USER,
        "rest_api_password": process.env.ZUORA_API_PASS,
        "rest_api_url": process.env.ZUORA_API_URL
    };
    const eloquaConfig = {
        "authorize_uri": process.env.ELOQUA_AUTHORIZE_URL,
        "token_uri": process.env.ELOQUA_TOKEN_URL,
        "rest_api_url": process.env.ELOQUA_API_URL,
        "rest_client_id": process.env.ELOQUA_CLIENT_ID,
        "rest_secret": process.env.ELOQUA_CLIENT_SECRET,
        "rest_api_company": process.env.ELOQUA_API_COMPANY,
        "rest_api_user": process.env.ELOQUA_API_USER,
        "rest_api_password": process.env.ELOQUA_API_PASS
    };
    var body = event || {};
    try {
        if (event.body || event.queryStringParameters) {
            body = Object.assign(event.body || {}, event.queryStringParameters || {});
        }
    } catch(e) {
        console.log(e);
        callback(e, {
            'statusCode': 500,
            'headers': { 
                'Content-Type': 'application/json',
                'Access-Control-Allow-Origin': '*'
            },
            'body': JSON.stringify({'Error': e.message})
        })
        return;
    }
    
    // TODO: add if statement for creating the import template
    // TODO: add an entry point for Zuora subscription callout to update single records in eloqua?
    return zuoraExport.getZuoraAccounts(body, zuoraConfig)
        .then(accounts => eloquaUpload.bulkUploadEloqua(accounts, eloquaConfig, body.instanceId, body.executionId))
        .then(() => callback(null, {
            'statusCode': 200,
            'headers': { 'Content-Type': 'text/plain' },
            'body': 'Success!'
        }))
        .catch(e => callback(e, {
            'statusCode': 500,
            'headers': { 'Content-Type': 'text/plain' },
            'body': 'Error: ' + e.message
        }))
}
module.exports = {
    handler
};


test notify entry point?


In [ ]:
var sinon = require('sinon');
var assert = require('assert');
var importer = require('../Core');
var bundle = importer.import('notify entry point');
var zuoraExport = importer.import('zuora account service');
var eloquaUpload = importer.import('bulk upload eloqua');

var sandbox = sinon.createSandbox();

describe('content notify entry point', () => {
    
    afterEach(() => {
        sandbox.restore();
    })
    
    it('should call zuora export', () => {
        const callback = sinon.spy();
        const dummyBody = {
        }
        
        const requestStub = sandbox.stub(zuoraExport, "getZuoraAccounts")
            .returns(Promise.resolve([]));
        sandbox.stub(eloquaUpload, "bulkUploadEloqua")
            .returns(Promise.resolve(true));
        
        return bundle.handler(dummyBody, null, callback)
            .then(() => {
                assert(callback.calledOnce);
                const stubCall = requestStub.getCall(0);
                assert.equal(stubCall.args[0], dummyBody);
            });
    })
    
    it('should call bulk upload', () => {
        const callback = sinon.spy();
        const dummyAccounts = [];
        
        sandbox.stub(zuoraExport, "getZuoraAccounts")
            .returns(Promise.resolve(dummyAccounts));
        const requestStub = sandbox.stub(eloquaUpload, "bulkUploadEloqua")
            .returns(Promise.resolve(true));
        
        return bundle.handler({}, null, callback)
            .then(() => {
                assert(callback.calledOnce);
                const stubCall = requestStub.getCall(0);
                assert.equal(stubCall.args[0], dummyAccounts);
            });
    })
})


zuora export month?



In [ ]:
var importer = require('../Core');
var renewalsQuery = importer.import('zuora renewals query');
var zuoraExport = importer.import('zuora to eloqua.ipynb[0]');

function getZuoraMonth(months, zuoraConfig) {
    if(!months) {
        months = 0;
    }
    var start = new Date();
    start.setMonth(start.getMonth() - months);
    start.setDate(1);
    start.setHours(0, 0, 0);
    var end = new Date();
    end.setMonth(end.getMonth() + months + 1);
    end.setDate(1);
    end.setHours(0, 0, 0);
    if(start.getMonth() > end.getMonth()) {
        end.getFullYear(end.getFullYear() + 1)
    }
    
    return zuoraExport.createBulkExportJob(renewalsQuery.getQuery(start.toString(), end.toString()), zuoraConfig)
        .then(exportId => zuoraExport.getBulkExportJobStatus(exportId, zuoraConfig))
        .then(fileId => zuoraExport.getBulkExportFile(fileId, zuoraConfig))
        .then(r => zuoraExport.csvToJson(r))
}
module.exports = {
    getZuoraMonth
};


test zuora export month?


In [ ]:
var assert = require('assert');
var sinon = require('sinon');
var exporter = importer.import('zuora export month');
var zuoraExport = importer.import('zuora to eloqua.ipynb[0]');
var renewalsQuery = importer.import('zuora renewals query');

var sandbox = sinon.createSandbox();
var zuoraConfig = {
    "rest_api_user":"devteam@fakepage.com",
    "rest_api_password":"pass",
    "rest_api_url": "http://localhost:18888"
};

describe('zuora export month', () => {
    
    afterEach(() => {
        sandbox.restore();
    })
    
    it('should get the query', () => {
        const dummyQuery = 'test query';
        
        sandbox.stub(zuoraExport, "createBulkExportJob").returns(Promise.resolve(''));
        sandbox.stub(zuoraExport, "getBulkExportJobStatus");
        sandbox.stub(zuoraExport, "getBulkExportFile");
        sandbox.stub(zuoraExport, "csvToJson").returns(Promise.resolve([]));
        const queryStub = sandbox.stub(renewalsQuery, "getQuery")
            .returns({ Query: dummyQuery });

        return exporter.getZuoraMonth(0, zuoraConfig).then(result => {
                assert.equal(result.length, 0);
                assert(queryStub.calledOnce, 'getQuery should only be called once');
                const stubCall = queryStub.getCall(0);
                assert.equal(new Date(stubCall.args[0]).getMonth(), (new Date()).getMonth());
        });
    })
    
    it('should call bulk export service', () => {
        const dummyQuery = 'test query';
        
        const exportStub = sandbox.stub(zuoraExport, "createBulkExportJob").returns(Promise.resolve(''));
        sandbox.stub(zuoraExport, "getBulkExportJobStatus");
        sandbox.stub(zuoraExport, "getBulkExportFile");
        sandbox.stub(zuoraExport, "csvToJson").returns(Promise.resolve([]));
        sandbox.stub(renewalsQuery, "getQuery")
            .returns({ Query: dummyQuery });

        return exporter.getZuoraMonth(0, zuoraConfig).then(result => {
                assert.equal(result.length, 0);
                assert(exportStub.calledOnce, 'createBulkExportJob should only be called once');
                const stubCall = exportStub.getCall(0);
                assert.equal(stubCall.args[0].Query, dummyQuery);
        });
    })
    
    it('should convert csv dump to json', () => {
        const dummyCsv = 'this,is,a,test\n1,2,3,4';
        
        sandbox.stub(zuoraExport, "createBulkExportJob").returns(Promise.resolve(''));
        sandbox.stub(zuoraExport, "getBulkExportJobStatus");
        sandbox.stub(zuoraExport, "getBulkExportFile").returns(Promise.resolve(dummyCsv));
        sandbox.stub(renewalsQuery, "getQuery").returns({ Query: 'test query' });
        
        return exporter.getZuoraMonth(0, zuoraConfig).then(result => {
                assert.equal(result[0].test, 4);
        });
    })
})


zuora account service?


In [ ]:
var zuoraExport = importer.import('zuora export service');
var request = importer.import('request polyfill');

module.exports = {
    getZuoraAccounts,
    zuoraQuery
};

function zuoraQuery(query, zuoraConfig) {
    return request.request({
        followAllRedirects: true,
        uri: zuoraConfig.rest_api_url + '/action/query',
        json: {
            queryString: query
        },
        method: 'POST',
        headers: zuoraExport.getAuthHeaders(zuoraConfig)
    });
}

function getContact(email, zuoraConfig) {
    return module.exports.zuoraQuery(`
SELECT AccountId, PersonalEmail, WorkEmail
FROM Contact
WHERE PersonalEmail LIKE '${email}'
OR WorkEmail LIKE '${email}'`, zuoraConfig)
}

function getAccountById(accountId, zuoraConfig) {
    return module.exports.zuoraQuery(`
SELECT Id, Status, Name, Currency, DefaultPaymentMethodId
FROM Account
WHERE Id='${accountId}'`, zuoraConfig);
}

function getPaymentMethod(paymentId, zuoraConfig) {
    return module.exports.zuoraQuery(`
SELECT CreditCardMaskNumber
FROM PaymentMethod
WHERE Id='${paymentId}'`, zuoraConfig);
}

function getAccountLast4Digits(email, zuoraConfig) {
    return getContact(email, zuoraConfig)
        .then(r => getAccountById(r.body.records[0].AccountId, zuoraConfig))
        .then(r => getPaymentMethod(r.body.records[0].DefaultPaymentMethodId, zuoraConfig))
        .then(r => {
            return {
                "EmailAddress": email,
                "Last4DigitsOfCard": r.body.records[0].CreditCardMaskNumber
            }
        })
}

function getZuoraAccounts(notifyRequest, zuoraConfig) {
    return Promise.all(notifyRequest.items.map(c => {
        return getAccountLast4Digits(c.EmailAddress, zuoraConfig);
    }))
}


test zuora account service?


In [ ]:
var sinon = require('sinon');
var assert = require('assert');
var importer = require('../Core');
var request = importer.import('request polyfill');
var accounts = importer.import('zuora account service');

var sandbox = sinon.createSandbox();
var zuoraConfig = {
    "rest_api_user":"devteam@fakepage.com",
    "rest_api_password":"pass",
    "rest_api_url": "http://localhost:18888"
};

describe('zuora account service', () => {
    afterEach(() => {
        sandbox.restore();
    })
    
    it('should call zuora query', () => {
        const dummyEmail = 'zuora-test@swiftipage.com'
        
        const queryStub = sandbox.stub(request, 'request').returns(Promise.resolve({ body: {records: [{}]} }))
        
        return accounts.getZuoraAccounts({
            items: [{
                 'EmailAddress': dummyEmail
            }]
        }, zuoraConfig).then(r => {
            assert(r);
            const queryCall = queryStub.getCall(0);
            assert(queryCall.args[0].json.queryString.includes(dummyEmail));
        });
    })
    
})


bulk upload eloqua?


In [ ]:
var assert = require('assert');
var importer = require('../Core');
var eloquaImport = importer.import('eloqua import service');
var eloquaObjects = importer.import('eloqua existing import');

module.exports = {
    bulkUploadEloqua,
};

function bulkUploadEloqua(accounts, eloquaConfig, instanceId, executionId) {
    var eloquaToken, importUri;
    return eloquaImport.eloquaOauth(eloquaConfig)
        .then(token => {
            eloquaToken = token;
            assert(token.expires > (new Date()).getTime());
            if(instanceId) {
                return eloquaImport.createInstanceDefinition(instanceId, executionId, eloquaToken, eloquaConfig)
            } else {
                // get custom data object ID from API
                var existingId;
                return eloquaObjects.getCustomDataObject(eloquaToken, eloquaConfig)
                    .then(existing => {
                        if(!existing) {
                            throw new Error('Eloqua custom data object not configured.')
                        }
                        existingId = existing.uri.split('/').pop();
                        return eloquaObjects.getImportDefinitions(existing.uri, eloquaToken, eloquaConfig);
                    })
                    .then(importUri => importUri
                          ? importUri
                          : eloquaImport.createImportDefinition(existingId, eloquaToken, eloquaConfig))
            }
        })
        .then(r => {
            importUri = r;
            return eloquaImport.startBulkImportData(accounts, importUri, eloquaToken, eloquaConfig);
        })
        .then(() => eloquaImport.completeBulkImportSync(importUri, eloquaToken, eloquaConfig))
        .then(() => accounts)
        .catch((e) => console.log(e))
}

if(typeof $$ !== 'undefined') {
    $$.async();
    bulkUploadEloqua()
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e))
}


test bulk upload eloqua?


In [ ]:
var assert = require('assert');
var sinon = require('sinon');
var importer = require('../Core');
var dataImporter = importer.import('bulk upload service');
var eloquaImport = importer.import('eloqua import service');
var eloquaObjects = importer.import('eloqua existing import');
var {
    getEloquaConfig,
    getOauthToken
} = importer.import('eloqua import blueprints');

var eloquaConfig = getEloquaConfig();
var eloquaToken = getOauthToken();
var sandbox = sinon.createSandbox();

describe('eloqua bulk upload', () => {
    
    afterEach(() => {
        sandbox.restore();
    })

    it('should call oauth', () => {
        const requestStub = sandbox.stub(eloquaImport, "eloquaOauth").returns(Promise.resolve(eloquaToken));
        sandbox.stub(eloquaImport, "createImportDefinition");
        sandbox.stub(eloquaImport, "startBulkImportData");
        sandbox.stub(eloquaObjects, "getImportDefinitions");
        sandbox.stub(eloquaObjects, "getCustomDataObject").returns(Promise.resolve({uri: '/customObject/1234'}));
        sandbox.stub(eloquaImport, "completeBulkImportSync").returns(Promise.resolve());
        
        return dataImporter.bulkUploadEloqua({}, eloquaConfig)
            .then(() => {
                const stubCall = requestStub.getCall(0);
                assert.equal(stubCall.args[0], eloquaConfig);
            })
    })
    
    it('should call import data', () => {
        sandbox.stub(eloquaImport, "eloquaOauth").returns(Promise.resolve(eloquaToken));
        sandbox.stub(eloquaImport, "createImportDefinition").returns(Promise.resolve({ body: {uri: '/imports/1234'} }));
        sandbox.stub(eloquaImport, "startBulkImportData").returns(Promise.resolve({ body: {status: 'success'} }));
        sandbox.stub(eloquaObjects, "getImportDefinitions");
        sandbox.stub(eloquaObjects, "getCustomDataObject").returns(Promise.resolve({uri: '/customObject/1234'}));
        const requestStub = sandbox.stub(eloquaImport, "completeBulkImportSync").returns(Promise.resolve({ body: {uri: '/imports/1234'} }));
        
        return dataImporter.bulkUploadEloqua({}, eloquaConfig)
            .then(() => {
                const stubCall = requestStub.getCall(0);
                assert.equal(stubCall.args[1], eloquaToken);
            })
    })
    
    it('should import temporary for each email from Content', () => {
        sandbox.stub(eloquaImport, "eloquaOauth").returns(Promise.resolve(eloquaToken));
        sandbox.stub(eloquaImport, "createInstanceDefinition").returns(Promise.resolve({ body: {uri: '/imports/1234'} }));
        sandbox.stub(eloquaImport, "startBulkImportData").returns(Promise.resolve({ body: {status: 'success'} }));
        const requestStub = sandbox.stub(eloquaImport, "completeBulkImportSync").returns(Promise.resolve({ body: {uri: '/imports/1234'} }));
        
        return dataImporter.bulkUploadEloqua({}, eloquaConfig, 'instance123', 'execution123')
            .then(() => {
                const stubCall = requestStub.getCall(0);
                assert.equal(stubCall.args[1], eloquaToken);
            })
    })
    
    it('should import an existing definition', () => {
        sandbox.stub(eloquaImport, "eloquaOauth").returns(Promise.resolve(eloquaToken));
        sandbox.stub(eloquaImport, "createImportDefinition").returns(Promise.resolve({ body: {uri: '/imports/1234'} }));
        sandbox.stub(eloquaImport, "startBulkImportData").returns(Promise.resolve({ body: {status: 'success'} }));
        sandbox.stub(eloquaObjects, "getImportDefinitions").returns(Promise.resolve({uri: '/customObject/1234'}));
        sandbox.stub(eloquaObjects, "getCustomDataObject").returns(Promise.resolve({uri: '/customObject/1234'}));
        const requestStub = sandbox.stub(eloquaImport, "completeBulkImportSync").returns(Promise.resolve({ body: {uri: '/imports/1234'} }));
        
        return dataImporter.bulkUploadEloqua({}, eloquaConfig)
            .then(() => {
                const stubCall = requestStub.getCall(0);
                assert.equal(stubCall.args[1], eloquaToken);
            })
    })
})


sync zuora eloqua end to end?



In [ ]:

describe('zuora to eloqua', () => {
    beforeEach(() => {
    })
    
    it('should export a month of zuora data', () => {
    })
    
    it('should match all products in zuora catalog', () => {
    })
    
    it('should transfer data end-to-end', () => {
    })
    
    /*
    TODO: 
Start a trial through portal
Add / remove support
ACC / distributor updates
Make a purchase
Purchase expires
Change quantity
Cancel subscription
Change quantity within 30 days of renewal
Change quantity within 60 days of renewal
AU/NZ renewals
ACC have eternally free terms
Renew with new subscription, same account
Renew new subscription, same email
*/
    
})


calculate price?


In [ ]:
// given a list of subscription IDs and products, calculate the new subscription total using a catalog export and compare with the preview API

// returns [account number, subtotal, previous discount]
function calculatePrice(subscription, products) {
    const rpcs = _.groupBy(subscription, r => r['RatePlanCharge.Id']);
    const charges = Object.keys(rpcs).map(k => _.sortBy(rpcs[k], r => r['RatePlanCharge.Version']).pop());
    var subtotal = 0;
    var discount;
    var quantity = 0;
    charges
        // TODO: escelate this to mapDataToFields function?
        .filter(c => !(c['ProductRatePlan.Name'] || '').toLowerCase().includes('perpetual'))
        .forEach(charge => {
            const product = products.filter(p => p['id'] === charge['Product.Id'])[0];
            const productRatePlan = product.productRatePlans.filter(p => p['id'] === charge['ProductRatePlan.Id'])[0];
            // select correct price plan for the item
            const productCharge = productRatePlan.productRatePlanCharges.filter(p => p['id'] === charge['ProductRatePlanCharge.Id'])[0];
            const pricing = productCharge.pricing.filter(p => p['currency'] === charge['Account.Currency'])[0];
        
        
            if((charge['ProductRatePlan.Name'] || '').toLowerCase().includes('discount')
                                    || (charge['ProductRatePlan.Name'] || '').toLowerCase().includes('volume')) {
                // TODO: add handler for quantity based discounts?
                discount = pricing.discountPercentage / 100;
            } else {
                const price = pricing.tiers === null
                    ? pricing.price
                    : pricing.tiers.filter(t => charge['RatePlanCharge.Quantity'] >= t['startingUnit']
                                           && charge['RatePlanCharge.Quantity'] <= t['endingUnit'])[0];
                if(typeof price === 'undefined') {
                    throw new Error('unknown rate plan component');
                }
                quantity += parseInt(charge['RatePlanCharge.Quantity']);
                subtotal += parseInt(charge['RatePlanCharge.Quantity']) * price;
            }
        });
    const discounts = charges.filter(c => (c['ProductRatePlan.Name'] || '').toLowerCase().includes('discount') > -1
                                    || (c['ProductRatePlan.Name'] || '').toLowerCase().includes('volume') > -1)
                             .filter(c => !(c['ProductRatePlan.Name'] || '').toLowerCase().includes('diamond')
                                    && !(c['ProductRatePlan.Name'] || '').toLowerCase().includes('distribution'));
    assert(!isNaN(subtotal), 'not a number! ' + JSON.stringify(charges, null, 4))
    //assert(discounts.length <= 1, 'multiple discounts! ' + JSON.stringify(discounts, null, 4))
    return [subscription[0]['Account.AccountNumber'], subtotal, discount, quantity];
}
module.exports = calculatePrice;


calculate price test?


In [ ]:
var _ = require('lodash');
var assert = require('assert');
var xlsx = require('xlsx');
var importer = require('../Core');
var fs = require('fs');

var getZuoraMonth = importer.import('zuora export month');
var calculatePrice = importer.import('calculate price');
var {getCatalog} = importer.import('zuora to eloqua.ipynb[0]');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE || '';
var zuoraConfig = JSON.parse(fs.readFileSync(PROFILE_PATH + '/.credentials/zuoraRest_production.json').toString().trim());

function filterROR(accountROR) {
    return [
            '4001372618',
            '4000919381',
            '411183297',
            '4001358862',
            '4000919006',
            '4000919116',
            '4001372618',
            '411182712',
            '411183101',
            '4001635120',
            '4000919342',
            '4000919068'].filter(ror => (accountROR || '').includes(ror)).length > 0
}

function rorsToAccounts(records) {
    return records.filter(a => a['Account.resellerofRecord__c'])
        .filter(a => filterROR(a['Account.resellerofRecord__c']))
        .map(a => a['Account.AccountNumber']);
}

function totalFilteredRecords(zuoraRecords) {
    const rors = rorsToAccounts(zuoraRecords);
    console.log(rors.length + ' ROR accounts with renewing subscriptions out of ' + zuoraRecords.length + ' records in the dump');
    const recordsToValidate = zuoraRecords
        .filter(s => {
            // TODO: graduate these filters to the main export / import?
            return new Date(s['Subscription.TermEndDate']).getTime() < (new Date('4/1/2018')).getTime()
                && new Date(s['Subscription.TermEndDate']).getTime() >= (new Date('2/27/2018')).getTime()
                && s['Account.Currency'] === 'USD'
                && s['RatePlanCharge.BillingPeriod'] === 'Annual' 
                && !s['ProductRatePlan.Name'].toLowerCase().includes('pro')
        })
    const recordsWithRors = recordsToValidate
        .filter(s => rors.includes(s['Account.AccountNumber']));
    console.log(recordsToValidate.length + ' zuora records to validate');
    console.log(recordsWithRors.length + ' of those are RORs and will also be excluded');
    console.log((recordsToValidate.length - recordsWithRors.length) + ' total');
    return recordsToValidate
        .filter(s => !rors.includes(s['Account.AccountNumber']))
}

function accountTotals(zuoraRecords) {
    const uniqueIds = _.groupBy(zuoraRecords, r => r['Account.Id']);
    console.log(Object.keys(uniqueIds).length + ' unique accounts with expiring subscriptions in February');
    const totals = Object.keys(uniqueIds)
        .map(accountId => {
            return calculatePrice(uniqueIds[accountId], catalog);
        });
    return totals;
}

function verifyMissing(worksheet, totals) {
    
    // TODO: compare with zuora preview API and Jacob's spreadsheet
    const accountIds = totals.map(t => t[0]);
    const worksheetIds = worksheet.map(t => t['Account Number']);
    const missing = worksheet.filter(a => !accountIds.includes(a['Account Number']));
    console.log(missing.length + ' accounts in worksheet, not in zuora export: ');
    //console.log(missing.slice(0, 20));
    const missingZuora = accountIds.filter(a => !worksheetIds.includes(a));
    console.log(missingZuora.length + ' accounts in zuora, missing from worksheet: ');
    console.log(missingZuora.slice(0, 20));
    const verifiableWorksheet = worksheet.filter(a => !missing.includes(a));
    const verifiableTotals = totals.filter(t => worksheetIds.includes(t[0]));
    console.log(verifiableWorksheet.length + ' = ' + verifiableTotals.length + ' verifiable records')
    return {verifiableWorksheet, verifiableTotals};
}

function validateWorksheet(calculatedTotals, zuoraRecords) {
    var workbook = xlsx.readFile(PROFILE_PATH + '/Documents/Marketing_File_Mar_.xlsx');
    var worksheet = xlsx.utils.sheet_to_json(workbook.Sheets['Marketing_File_Mar_']);
    console.log(worksheet.length + ' rows in worksheet');
    const worksheetUSD = worksheet.filter(t => t['Currency'] === 'USD');
    console.log(worksheetUSD.length + ' rows are USD');
    const worksheetFiltered = worksheetUSD.filter(t => filterROR(t['ROR Number'] || '') === false);
    console.log(worksheetFiltered.length + ' rows are USD and not ROR');
    
    const {verifiableWorksheet, verifiableTotals} = verifyMissing(worksheetFiltered, calculatedTotals);
    
    const correct = verifiableWorksheet.filter(a => {
        const realTotal = parseFloat((a['Total 1:'] || '').replace(/[\$,\s]/ig, ''));
        const newTotal = parseFloat((a['Total'] || '').replace(/[\$,\s]/ig, ''));
        const oldTotal = verifiableTotals.filter(t => t[0] === a['Account Number'])[0] || [];
        // TODO: decide what to do with discounts?
        if(a['Account Number'] === 'A00191395') {
            console.log(newTotal);
            console.log(oldTotal);
            console.log(oldTotal[1] - (oldTotal[2] ? (oldTotal[1] * oldTotal[2]) : 0))
        }
        return realTotal === oldTotal[1]
            || newTotal === oldTotal[1]
            || newTotal === oldTotal[1] - (oldTotal[2] ? (oldTotal[1] * oldTotal[2]) : 0)
            || newTotal === oldTotal[1] - (oldTotal[1] * .05);
    });
    const incorrect = verifiableWorksheet.filter(a => !correct.includes(a));
    console.log(incorrect.length + ' incorrect, correct: ' + correct.length + ' out of ' + verifiableTotals.length
                + ' - ' + Math.round(correct.length / verifiableTotals.length * 100) + '%');
    // TODO: fix this
    console.log(incorrect.length + ' + ' + correct.length + ' = ' + (incorrect.length + correct.length));
    // TODO: calculate perpetual price for previousinvoice price comparison?
    // TODO: how to handle australia?
    // TODO: how to handle not in worksheet?
    return {correct, incorrect};
}

var catalog, records;
function compareRecordsCatalog() {
    return (typeof catalog !== 'undefined' ? Promise.resolve(catalog) : getCatalog(zuoraConfig))
        .then(r => (catalog = r))
        .then(() => typeof records !== 'undefined' ? Promise.resolve(records) : getZuoraMonth(6, zuoraConfig))
        .then(r => {
            // filter out the records we aren't validating
            records = r;
            const recordsToValidate = totalFilteredRecords(records);
            const zuoraTotals = accountTotals(recordsToValidate);
            
            const {correct, incorrect} = validateWorksheet(zuoraTotals);
        
            const displayIncorrect = incorrect.map(a => Object.assign(a, {
                incorrect: zuoraTotals.filter(t => t[0] === a['Account Number'])[0] || [],
                subscription: JSON.stringify(recordsToValidate.filter(r => r['Account.AccountNumber'] === a['Account Number']).map(s => s['ProductRatePlan.Name'])),
//                    multiplePrimary:  TODO: check if selecting one product fixes the price
            }))
            const incorrectMultipleSubs = displayIncorrect.filter(a => {
                const subs = recordsToValidate.filter(r => r['Account.AccountNumber'] === a['Account Number']);
                const productGroups = _.groupBy(subs, e => e['ProductRatePlan.Name']);
                return Object.keys(productGroups).length === 0 || Object.keys(productGroups).filter(k => productGroups[k].length > 1).length > 0;
            })
            const verifiableTotal = (correct.length + incorrect.length) - incorrectMultipleSubs.length;
            console.log(incorrectMultipleSubs.length + ' incorrect due to multiple subscriptions, correct minus multiple: '
                        + correct.length + ' out of ' + verifiableTotal + ' - ' + Math.round(correct.length / verifiableTotal * 100) + '%');
            const unaccounted = displayIncorrect.filter(a => !incorrectMultipleSubs.includes(a));
            console.log(unaccounted.length + ' unaccounted for');
            console.log('incorrect sample (' + displayIncorrect.length + '): ');
            console.log(unaccounted); //.map(r => r.subscription)
            
            return zuoraTotals;
        });
}

if(typeof $$ !== 'undefined') {
    $$.async();
    // TODO: pull zuora product catalog
    compareRecordsCatalog()
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e))
    // TODO: this takes to long to download, describe blocks?
}




readme?

# Install

http://www.oracle.com/technetwork/java/javase/downloads/index.html

https://jenkins.io/doc/pipeline/tour/getting-started/

https://nodejs.org

http://docs.aws.amazon.com/cli/latest/userguide/installing.html


Or use Docker

https://github.com/jenkinsci/docker

`docker run -p 8080:8080 -p 50000:50000 -n jenkins jenkins/jenkins:lts`


`docker exec -it -u root jenkins wget -O - https://deb.nodesource.com/setup_8.x | bash && apt-get install -y nodejs zip aws`


# Setup

Create a pipeline and copy the Jenkinsfile, or import from Github

After 61 build tries you should have a screen that looks all green!


# Test

`npm run test`

# Deploy

Create an access key for AWS: https://console.aws.amazon.com/iam/home?nc2=h_m_sc#/security_credential

http://docs.aws.amazon.com/AWSGettingStartedContinuousDeliveryPipeline/latest/GettingStarted/CICD_Jenkins_Pipeline.html

`aws configure`

`aws lambda update-function-code --function-name eloqua_test --zip-file fileb://index.zip`



Get account information

In [ ]:
var _ = require('lodash');
var fs = require('fs');
var importer = require('../Core');
var { request } = importer.import('request polyfill');
var { getAuthHeaders } = importer.import('zuora export service');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE || '';
var zuoraConfig = JSON.parse(fs.readFileSync(PROFILE_PATH + '/.credentials/zuoraRest_production.json').toString().trim());

function zuoraQuery(query) {
    return request({
        followAllRedirects: true,
        uri: zuoraConfig.rest_api_url + '/action/query',
        json: {
            queryString: query
        },
        method: 'POST',
        headers: getAuthHeaders(zuoraConfig)
    });
}

function getContact(email) {
    return zuoraQuery(`SELECT AccountId, PersonalEmail, WorkEmail FROM Contact WHERE PersonalEmail LIKE '${email}' OR WorkEmail LIKE '${email}'`)
}

function getContactByAccount(accountId) {
    return zuoraQuery(`SELECT AccountId, PersonalEmail, WorkEmail FROM Contact WHERE AccountId='${accountId}'`)
}

function getAccountById(accountId) {
    return zuoraQuery(`SELECT Id, Status, Name, Currency, DefaultPaymentMethodId FROM Account WHERE Id='${accountId}'`);
}

function getAccount(accountNumber) {
    return zuoraQuery(`SELECT Id, Status, Name, Currency, DefaultPaymentMethodId FROM Account WHERE AccountNumber='${accountNumber}'`);
}

function getSubscription(accountId) {
    return zuoraQuery(`SELECT Id, Status, TermEndDate FROM Subscription WHERE AccountId='${accountId}'`);
}

function getPaymentMethod(paymentId) {
    return zuoraQuery(`SELECT CreditCardMaskNumber FROM PaymentMethod WHERE Id='${paymentId}'`);
}

function getRatePlans(subscriptionIds) {
    return zuoraQuery(`SELECT Id, Name, SubscriptionId FROM RatePlan WHERE SubscriptionId='${subscriptionIds.join('\', OR SubscriptionId=\'')}'`);
}

if(typeof $$ !== 'undefined') {
$$.async();
    var accountId, paymentId;
    getContact('flyfisher8008@yahoo.com')
        .then(r => {
            console.log(r.body.records);
            return getAccountById(r.body.records[0].AccountId)
        })
        .then(r => {
            console.log(r.body.records);
            accountId = r.body.records[0].Id;
            paymentId = r.body.records[0].DefaultPaymentMethodId
            return getSubscription(accountId)
        })
        .then(r => {
            console.log(r.body.records);
            return getRatePlans(r.body.records.map(r => r.Id));
        })
        .then(r => {
            console.log(_.groupBy(r.body.records, r => r.SubscriptionId))
            return getPaymentMethod(paymentId);
        })
        .then(r => {
            console.log(r.body.records[0].CreditCardMaskNumber)
            return getContactByAccount(accountId);
        })
        .then(r => r.body.records)
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e))
}
